In [1]:
import sys
sys.path.append('/Users/dan/Code/LLM/llm_researcher')

In [2]:
from llm_researcher.master.agent import LLMResearcher
from llm_researcher.master.functions import (add_source_urls, extract_headers, table_of_contents)
from llm_researcher.utils.write_md import write_md_to_pdf

In [3]:
async def get_subtopic_report(query, subtopic, agent, role, global_context, global_urls, existing_headers) -> tuple:
    current_subtopic_task = subtopic.get("task")
    subtopic_assistant = LLMResearcher(
        query = current_subtopic_task,
        report_type = "subtopic_report",
        websocket = None,
        parent_query = query,
        visited_urls = global_urls,
        agent =  agent,
        role = role
    )

    # The subtopics should start research from the context gathered till now
    subtopic_assistant.context = list(set(global_context))

    # Conduct research on the subtopic
    await subtopic_assistant.conduct_research()

    # Here the headers gathered from previous subtopic reports are passed to the write report function
    # The LLM is later instructed to avoid generating any information relating to these headers as they have already been generated
    subtopic_report = await subtopic_assistant.write_report(existing_headers)

    # Update context of the global context variable
    global_context = list(set(subtopic_assistant.context))
    # Update url list of the global list variable
    global_urls.update(subtopic_assistant.visited_urls)

    # After a subtopic report has been generated then append the headers of the report to existing headers
    existing_headers.append(
        {
            "subtopic task": current_subtopic_task,
            "headers": extract_headers(subtopic_report),
        }
    )
    return { "topic": current_subtopic_task, "report": subtopic_report }
        

In [4]:


async def run(query,report_type):
    source_urls = []
    subtopics = []
    existing_headers = []
    global_context = []
    global_urls = set(source_urls)

    main_task_assistant = LLMResearcher(query=query, report_type=report_type)
    await main_task_assistant.conduct_research()
    global_context = main_task_assistant.context
    global_urls = main_task_assistant.visited_urls

    subtopics = await main_task_assistant.get_subtopics()
   

    subtopic_reports = []
    subtopics_report_body = ""
    for subtopic in subtopics:
        result = await get_subtopic_report(query, subtopic, main_task_assistant.agent, main_task_assistant.role, global_context, global_urls, existing_headers)
        if result["report"]:
            subtopic_reports.append(result)
            subtopics_report_body += "\n\n\n" + result["report"]

    main_task_assistant.visited_urls.update(global_urls)

    introduction = await main_task_assistant.write_introduction()
    toc = table_of_contents(subtopics_report_body)
    report_with_references = add_source_urls(subtopics_report_body, main_task_assistant.visited_urls)
    report = f"{introduction}\n\n{toc}\n\n{report_with_references}"

    return report

In [5]:
query = "Research Lung Cancer and provide information on mortality rates, cancer stages, and symptoms that might be observed at each stage."
report_type = "research_report"

report = await run(query, report_type)
await write_md_to_pdf(report)
report


Successfully imported function tavily_search
🔎 Running research for 'Research Lung Cancer and provide information on mortality rates, cancer stages, and symptoms that might be observed at each stage.'...
🩺 Medical Research Agent
🧠 I will conduct my research based on the following queries: ['Lung cancer mortality rates 2024', 'Lung cancer stages and symptoms 2024', 'Lung cancer symptoms by stage 2024', 'Research Lung Cancer and provide information on mortality rates, cancer stages, and symptoms that might be observed at each stage.']...

🔎 Running research for 'Lung cancer mortality rates 2024'...
✅ Adding source url to research: https://seer.cancer.gov/statfacts/html/lungb.html

✅ Adding source url to research: https://acsjournals.onlinelibrary.wiley.com/doi/full/10.3322/caac.21820

✅ Adding source url to research: https://www.singlecare.com/blog/news/lung-cancer-statistics/

✅ Adding source url to research: https://www.nature.com/articles/s41571-023-00798-3

✅ Adding source url to res

"# Comprehensive Analysis of Lung Cancer: Mortality Rates, Stages, and Symptoms\n\nLung cancer remains a significant global health challenge, being the leading cause of cancer death worldwide. This report delves into the latest research on lung cancer, focusing on mortality rates, cancer stages, and the symptoms observed at each stage.\n\n## Mortality Rates\n\nLung cancer is the foremost cause of cancer-related deaths in the United States, with a death rate of 32.4 per 100,000 men and women per year based on data from 2018–2022 ([SEER](https://seer.cancer.gov/statfacts/html/lungb.html)). The death rates vary significantly by race and sex. For instance, non-Hispanic Black males have the highest death rate at 46.7 per 100,000, while Hispanic females have the lowest at 11.1 per 100,000. The median age at death for lung cancer patients is 73 years.\n\nGlobally, the incidence and mortality trends of lung cancer reflect the historical patterns of tobacco smoking. Economically developed count

# Run steps individualy

In [ ]:
query = "Information on Horseback Riding Tours in Ireland"
report_type = "research_report"
 
main_task_assistant = LLMResearcher(query=query, report_type=report_type)
await main_task_assistant.conduct_research()
global_context = main_task_assistant.context
global_urls = main_task_assistant.visited_urls

In [ ]:
subtopics = [{'task': 'Overview of the Burning Man Flooding Incident'},
 {'task': 'Impact and Response to the Flooding'},
 {'task': "Reflections on Climate Change and Burning Man's Environmental Impact"}]
agent = "\ud83d\udcf0 News Analyst Agent",
role = "You are a well-informed AI news analyst assistant. Your main task is to compile detailed, insightful, unbiased, and well-organized news reports on recent events, including natural disasters, providing context, impact analysis, and relevant updates."
existing_headers = []

subtopic_reports = []
subtopics_report_body = ""
for subtopic in subtopics:
    result = await get_subtopic_report(query, subtopic, agent, role, global_context, global_urls, existing_headers)
    if result["report"]:
        subtopic_reports.append(result)
        subtopics_report_body += "\n\n\n" + result["report"]


In [ ]:
introduction = await main_task_assistant.write_introduction()
introduction

In [ ]:
toc = table_of_contents(subtopics_report_body)
report_with_references = add_source_urls(subtopics_report_body, main_task_assistant.visited_urls)
report = f"{introduction}\n\n{toc}\n\n{report_with_references}"
await write_md_to_pdf(report)
report

# ==================================================================================================

# ###################################################################

In [ ]:
query = "What happened in the latest burning man floods?"
report_type = "research_report"
researcher = LLMResearcher(query=query, report_type=report_type)
researcher.context =[
    "Source: https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193\nTitle: \nContent: STEPS FOR VETSNBC News Site MapHelpFollow NBC News news AlertsThere are no new alerts at this timeSearchSearchFacebookTwitterEmailSMSPrintWhatsappRedditPocketFlipboardPinterestLinkedinLatest StoriesTrump on TrialPoliticsU.S. NewsWorldBusinessSportsHealthNBC News TiplineCulture & TrendsEVENT ENDEDLast updated Sept. 4, 2023, 12:20 PM UTCBurning Man flooding keeps thousands stranded at Nevada site as authorities investigate 1 death Attendees at the countercultural music and arts festival were first advised to \u201cshelter in place\u201d and conserve food and water.PrintUpdated\u00a0Sept. 4, 2023, 12:20 PM UTCBy NBC NewsBurning Man attendees struggling to get homeBurning Man attendees are still struggling to escape from Black Rock City as around 70,000 remain stuck on muddy campgrounds after two days of heavy rain.\u201cThere is no cause for panic,\" CEO Marian Goodell told NBC News. Attendees at the countercultural music and arts festival were first advised to \u201cshelter in place\u201d and conserve food and water\n\nSource: https://www.cbsnews.com/news/burning-man-death-investigation-70000-stranded-nevada-desert-flooding/\nTitle: \nContent: Tents between puddles and mud on the grounds of the \"Burning Man\" festival on Sept. 3, 2023, in the Nevada desert. Tens of thousands of visitors to the desert festival were stranded due to heavy rainfall.\u00a0\n            \n                David Crane/picture alliance via Getty Images\n                          \nThe White House said Sunday that President Biden had been briefed on the flooding and \"administration officials are monitoring the situation and are in touch with state and local officials.\"\n\nSource: https://www.cnn.com/2023/09/05/us/burning-man-storms-shelter-exodus-tuesday/index.html\nTitle: \nContent: Matt Mills McKnight/Reuters\n    Why did the rain have such an impact on Burning Man?\n            After the first rains soaked campsites Friday, event leaders alerted attendees that driving was no longer allowed in the city. They halted entry and exit in the area, urged people to \u201cshelter in place,\u201d and conserve food and water.\n    \n            The remote area in northwest Nevada was hit with up to 0.8 inches \u2013 about twice the average September rainfall \u2013 in just 24 hours between Friday and Saturday morning. And more rain followed Sunday. \n    \nAmerican DJ Diplo performs during the 2023 Governors Ball Music Festival at Flushing Meadows Corona Park in New York City, on June 9, 2023.\nAngela Weiss/AFP/Getty Images\nDiplo hitchhiked ride out of rain-drenched Burning Man after walking miles 'through the mud' and actually made it to his DC concert\n\nSource: https://www.cbsnews.com/news/burning-man-death-investigation-70000-stranded-nevada-desert-flooding/\nTitle: \nContent: 02:33\nOne person died at the Burning Man festival this weekend as a severe bout of rain caused flooding in the remote Nevada desert where the event was being held, authorities said. This comes as tens of thousands of attendees remained stranded Sunday.\u00a0The Pershing County Sheriff's Office confirmed it is investigating the death in a news release issued Saturday, and said the family has been notified both of the death and the active probe. A spokesperson for the sheriff's office told CBS News on Sunday that the death occurred during the extreme rain that swept through parts of northwestern Nevada on Friday and Saturday, but not because of it. The earlier news release noted that the office \"is currently investigating a death which occurred during this rain event.\"\u00a0Echoing the release, the spokesperson declined to share additional information about the death or circumstances surrounding it because the sheriff's investigation was still ongoing.\n\nSource: https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193\nTitle: \nContent: Burning Man flooding keeps thousands stranded at Nevada site as authorities investigate 1 death IE 11 is not supported. For an optimal experience visit our site on another browser.SKIP TO CONTENTNBC News LogoTrump on TrialPoliticsU.S. NewsWorldBusinessSportsHealthNBC News TiplineCulture & TrendsShare & Save\u00a0\u2014My NewsManage ProfileEmail PreferencesSign OutSearchSearchProfile\u00a0My NewsSign Out\u00a0Sign InCreate your free profileSectionsU.S. NewsPoliticsWorldLocalBusinessSportsParis 2024 OlympicsInvestigationsCulture & TrendsHealthScienceTech & MediaWeatherVideo FeaturesPhotosNBC SelectDecision 2024NBC Asian AmericaNBC BLKNBC LatinoNBC OUTtvTodayNightly NewsMSNBCMeet the PressDatelineFeaturedNBC News NowNightly FilmsStay TunedSpecial FeaturesNewslettersPodcastsListen NowMore From NBCCNBCNBC.COMNBCU AcademyPeacockNEXT STEPS FOR VETSNBC News Site MapHelpFollow NBC News news AlertsThere are no new alerts at this\n\nSource: https://www.cnn.com/us/live-news/nevada-desert-burning-man-weather-rain-09-03-23/index.html\nTitle: \nContent: in place in the Black Rock Desert and conserve food, water and fuel, but some have chosen to hike to the nearest town. Officials have closed the road in and out of the festival to all but emergency vehicles.The remote event site in northwest Nevada was hit with 2 to 3 months\u2019 worth of rain in just 24 hours from Friday to Saturday morning. More is anticipated Sunday afternoon.Authorities are investigating a death \"which occurred during this rain event,\" according to the sheriff's office \u2014 though it is unclear if the death was related to the storms.share with Facebookshare with Twittershare with emailshare link12 PostsSort byLatestOldestDropdown arrow6:40 p.m. ET, September 3, 2023President Biden has been briefed on the situation at Burning Man festivalFrom CNN's Betsy KleinA still from a drone video shows vehicles trying to leave the Burning Man festival on Sunday, September 3. CNNPresident Joe Biden was briefed Sunday on the situation at the Burning Man festival in Black Rock Desert,\n\nSource: https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193\nTitle: \nContent: is no cause for panic,\u201d Marian Goodell told NBC News.Burning Man\u2019s estimated 70,000-plus attendees were isolated at the venue known as Black Rock City as rain returned Sunday and as closed roads, muddy campgrounds and\u00a0one reported death\u00a0darkened the day.Read the full story here.248d ago / 11:23 AM UTCCopiedShareThousands still stranded at Burning Man after torrential rain 248d ago / 10:54 AM UTCCopiedShareBurning Man revelers unfazed by deluge and deep mud ReutersThousands of Burning Man attendees partied hard on Sunday despite downpours that turned the Nevada desert where the annual arts and music festival takes place into a sea of sticky mud and led officials to order the multitudes to shelter in place.While hundreds escaped, others got stuck in the thick, gooey mud that coated the normally dusty lake bed where the festival occurs. Those remaining said there was plenty of food and drinks to keep the party going.\u201cWhen you get pushed to extremes, that\u2019s when the most fun happens,\u201d\n\nSource: https://www.cnn.com/us/live-news/nevada-desert-burning-man-weather-rain-09-03-23/index.html\nTitle: \nContent: and pushed it behind schedule, the team behind the building wrote on Instagram.Then more rain came during the festival itself, with 2 to 3 months\u2019 worth of rain falling on the remote event site in just 24 hours from Friday to Saturday morning. More is anticipated Sunday afternoon.share with Facebookshare with Twittershare with emailshare link12:25 p.m. ET, September 3, 2023Burning Man organizers working to provide cell service and medical resources as attendees shelter in placeFrom CNN's Nouran Salahieh\u00a0and\u00a0Emma TuckerBurning Man organizers say they are placing mobile cell trailers around the battered festival grounds, configuring their Wi-Fi system for public access, and sending buses to nearby Gerlach, Nevada, to pick up people who might have walked out of the desert and ferry them to Reno.Event staff members are also calling in more four-wheel-drive vehicles and all-terrain tires to help in the case of any medical or other urgent situations. Traffic in and out of the event grounds\n",
    "Source: https://cbs12.com/news/nation-world/flooding-strands-thousands-at-burning-man-festival-site-death-under-investigation-black-rock-desert-nevada-playa-severe-weather-conditions-bureau-of-land-management\nTitle: \nContent: Flooding strands thousands at Burning Man festival site, death under investigationPlease ensure Javascript is enabled for purposes ofwebsite accessibilityThu, 09 May 2024 16:19:05 GMT (1715271545895)Story Infinite Scroll - News3 v1.0.0 (common)56a10e3abb3a973e74c8109f39c0e8dd0d8529b8e201a25d3fc2cb16d17ece29beb4b95b8dfba983NewsWeatherRadarMapsWeather AppHurricane CenterMarine ForecastVideosEN ESPA\u00d1OLGame CenterWatch Now 88 Fri 95 Sat 92Flooding strands thousands at Burning Man festival site, death under investigationby Associated PressSun, September 3rd 2023 at 10:25 AMUpdated Sun, September 3rd 2023 at 6:14 PMIn this satellite photo provided by Maxar Technologies, an overview of Burning Man festival in Black Rock, Nev on Monday, Aug. 28, 2023.  (\u00a92023 Maxar Technologies via AP)RENO, Nev. (AP) \u2014 An unusual late-summer storm turned a week-long counterculture fest into a sloppy mess with tens of thousands of partygoers stuck in foot-deep mud and with no working toilets in the northern\n\nSource: https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html\nTitle: \nContent: 09:44\n                            \nNow playing\n - Source:\n                CNN\nSee More Videos\nCNN\n        \u00a0\u2014\u00a0\n    \n            Tens of thousands of people attending the Burning Man festival in the Nevada desert are being told to conserve food, water and fuel as they shelter in place in the Black Rock Desert after a heavy rainstorm pummeled the area, festival organizers said. \n    \n            Attendees saw their campsites transformed by thick, ankle-deep mud and organizers halted vehicles from traveling in or out of the festival after heavy rains started saturating the area Friday evening. Some festival-goers hiked miles to reach main roads while others hoped storms forecast to hit the area overnight wouldn\u2019t worsen conditions.\n\nSource: https://cbs12.com/news/nation-world/flooding-strands-thousands-at-burning-man-festival-site-death-under-investigation-black-rock-desert-nevada-playa-severe-weather-conditions-bureau-of-land-management\nTitle: \nContent: tens of thousands of partygoers stuck in foot-deep mud and with no working toilets in the northern Nevada desert, But some Burning Man revelers said Sunday that their spirits remained unbroken.\u201cWe are a little bit dirty and muddy but spirits are high. The party still going,\u201d said Scott London, a Southern California photographer, adding that the travel limitations offered \u201ca view of Burning Man that a lot of us don\u2019t get to see.\u201dThe annual gathering in the Black Rock Desert about 110 miles (177 kilometers) north of Reno attracts nearly 80,000 artists, musicians and activists for a mix of wilderness camping and avant-garde performances. Disruptions are part of the event's recent history: Organizers had to temporarily close entrances to the festival in 2018 due to dust storms, and the event was twice canceled altogether during the pandemic.More than a half-inch of rain fell at the festival site on Friday, the National Weather Service in Reno said, with more coming Saturday and Sunday,\n\nSource: https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html\nTitle: \nContent: Andrew Hyde\n            \u201cRain over the last 24 hours has created a situation that required a full stop of vehicle movement on the playa. More rain is expected over the next few days and conditions are not expected to improve enough to allow vehicles to enter the playa,\u201d the statement read.\n    \n            The festival, which began in 1986, is held each summer in Black Rock City \u2013 a temporary metropolis that is erected annually for the festival. The city comes complete with planning services, emergency, safety and sanitary infrastructure.\n    \n            It is best known for its concluding event, in which a large wooden symbol of a man is ignited. The event attracts tens of thousands each year and in the past, celebrities from Sean \u201cDiddy\u201d Combs to Katy Perry have attended.\n\nSource: https://cbs12.com/news/nation-world/flooding-strands-thousands-at-burning-man-festival-site-death-under-investigation-black-rock-desert-nevada-playa-severe-weather-conditions-bureau-of-land-management\nTitle: \nContent: RVs for sleeping, offering food and coffee,\u201d Barger said. \u201cI danced in foot-deep clay for hours to incredible DJs.\u201dEd Fletcher of Sacramento, a longtime Burning Man attendee, arrived in Black Rock City over a week ago to start setting up. When the rain hit, he and his campmates threw a party and \u201cdanced the night away\u201d in their muddy shoes.\u201cRadical self-reliance is one of the principles of Burning Man,\u201d he said. \u201cThe desert will try to kill you in some way, shape or form.\u201dThe Pershing County Sheriff\u2019s Office did not release the identity of the deceased person or the suspected cause of death but said it is being investigated.On their website, organizers encouraged participants to remain calm and suggested that the festival is built to endure conditions like the flooding. They said cellphone trailers were being dropped in several locations Saturday night and that they would be briefly opening up internet overnight. Shuttle buses were also being organized to take attendees to Reno from\n\nSource: https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html\nTitle: \nContent: A satellite view shows an overview of the 2023 Burning Man festival, in Black Rock Desert, Nevada on August 28, 2023.\nMaxar Technologies/Reuters\n            More than 70,000 people attend the weeklong event annually, which this year is being held from August 28 to September 4. It\u2019s unclear how many of those were stranded due to the weather. \n    \n            The city is expecting more showers overnight on Saturday, organizers said in a weather forecast update. The National Weather Service said showers and thunderstorms are expected to return Saturday evening and continue throughout Sunday, with temperatures ranging from highs in the 70s to a low overnight of 49 degrees. Labor Day, on Monday when the event is scheduled to end, forecasts show the area will heat up and dry out with clear skis and a high of 75 degrees.\n\nSource: https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html\nTitle: \nContent: Hannah Burhorn, a first-time attendee at the festival, told CNN in a phone interview Saturday the desert sand has turned into thick clay and puddles and mud are everywhere. People are wrapping trash bags and Ziploc bags around their shoes to avoid getting stuck, while others are walking around barefoot. \n    \n            \u201cIt\u2019s unavoidable at this point,\u201d she said. \u201cIt\u2019s in the bed of the truck, inside the truck. People who have tried to bike through it and have gotten stuck because it\u2019s about ankle deep.\u201d  \n    \n            The gate and airport into Black Rock City, a remote area in northwest Nevada, remain closed and no driving is allowed into or out of the city except for emergency vehicles, the organizers said on X, the social media platform formerly known as Twitter.\n    \n            \u201cDo not travel to Black Rock City! Access to the city is closed for the remainder of the event, and you will be turned around,\u201d one statement read.\n\nSource: https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html\nTitle: \nContent: Rainfall reports from the National Weather Service suggest up to 0.8 inches of rain fell in the area from Friday morning through Saturday morning \u2013 approximately two to three months of rainfall for that location this time of year. Even small rainfall totals can lead to flooding in the dry Nevada desert. \n    \n            Flood watches were in effect in northeast Nevada, to the east of Black Rock City. Those watches noted individual storms were producing up to one inch of rainfall, but higher totals \u2014 as much as 3 inches \u2014 would be possible through the weekend.\n    \n            The Bureau of Land Management, which has jurisdiction over the land the festival is held on, is advising people heading to Burning Man to \u201cturn around and head home,\u201d as roads remain closed in the area, according to a statement obtained by the Reno Gazette-Journal.\n    \nMud fills a Burning Man campsite after heavy rain in Nevada's Black Rock Desert on September 1, 2023.\nAndrew Hyde\n",
    "Source: https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/\nTitle: \nContent: Newsletters\nDonate\nFollow us\nThe Facts Newsletter\nSign up\nEnglish\nEspa\u00f1ol\nStand up for the facts!\nOur only agenda is to publish the truth so you can be an informed participant in democracy.\nWe need your help.\nMore Info\nI would like to contribute\nOne Time\nMonthly\nYearly\nJoin Now\nInstagram posts\nstated on September 3, 2023 in a video shared on Instagram:\n\u201cBurning Man was a national emergency because it was flooded.\u201d\nFacebook Fact-checks\nInstagram posts\nPeople leave the Burning Man festival in Nevada on Sept. 5, 2023. The traffic jam leaving Burning Man eased up considerably Tuesday as the exodus from the mud-caked Nevada desert entered another day. (AP)\nBy Madison Czopek\nSeptember 6, 2023\nThere were floods, there was mud, but Burning Man sparked no emergency declaration\nIf Your Time is short\nHeavy rain began falling Sept. 1 in Nevada\u2019s Black Rock Desert. The rain caused flooding at Burning Man, a counterculture festival that draws thousands of people to a makeshift city.\n\nSource: https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/\nTitle: \nContent: Our ruling\nAn Instagram video claimed \"Burning Man was a national emergency because it was flooded.\"\nHeavy rain caused flooding at Burning Man, triggering road closures and temporarily stranding thousands of attendees. But by early afternoon Sept. 4, people were able to exit the festival site.\nWe found no evidence the flooding triggered a national emergency declaration. We rate this claim False.\nRead About Our Process\nThe Principles of the Truth-O-Meter\nOur Sources\nInstagram post, Sept. 3, 2023\nTikTok post, Sept. 3, 2023\nCNN, Tens of thousands at Burning Man told to conserve water and food after heavy rains leave attendees stranded in Nevada desert, Sept. 2, 2023\nThe Reno Gazette Journal, Burning Man 2023: Cell towers coming; no estimate of when vehicles can resume travel, Sept. 3, 2023\nThe Reno Gazette Journal, Burning Man: Drugs suspected in man's death at Nevada festival, medical examiner says, Sept. 5, 2023\n\nSource: https://www.knpr.org/news-88-9-knpr/2023-12-27/after-flooding-burning-man-to-go-on-as-planned-in-2024\nTitle: \nContent: After flooding, Burning Man to go on as planned in 2024\nSearch Query\nShow Search\nNews\nKNPR's State of Nevada\nElection 2024\nPolitics\nEnvironment\nCrime\nArts, Culture and Dining\nBusiness\nEducation\nSports\nHealth\nMountain West News Bureau\nICYMI Fridays\nKNPR's State of Nevada\nElection 2024\nPolitics\nEnvironment\nCrime\nArts, Culture and Dining\nBusiness\nEducation\nSports\nHealth\nMountain West News Bureau\nICYMI Fridays\nDesert Companion\nAll Things\nCulture\nFood and Drink\nThe Guide\nDesert Air\nAll Things\nCulture\nFood and Drink\nThe Guide\nDesert Air\nClassical\nSupport\nWays to Give\nLog In\nWays to Give\nLog In\nBox Office\nAbout\nSubmit a news tip or press release\nPrograms\nStaff\nBoard of Directors\nEmployment Opportunities\nKNPR 88.9 FM Schedule\nKCNV 89.7 FM Schedule\nMore About Nevada Public Radio\nSubmit a news tip or press release\nPrograms\nStaff\nBoard of Directors\nEmployment Opportunities\nKNPR 88.9 FM Schedule\nKCNV 89.7 FM Schedule\nMore About Nevada Public Radio\nNewsletters\n instagram\n facebook\n\nSource: https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/\nTitle: \nContent: Newsweek, Is FEMA at Burning Man? Virus Outbreak Conspiracy Theory Spreads Online, Sept. 4, 2023\nThe Associated Press, Burning Man flooding strands tens of thousands at Nevada site; authorities are investigating 1 death, Sept. 3, 2023\nBBC, Thousands queue for hours to leave Burning Man festival, Sept. 5, 2023\nCBS News, Burning Man \"exodus operations\" begin as driving ban is lifted, organizers say, Sept. 4, 2023\nThe Associated Press, Burning Man flooding triggers false claims of Ebola outbreak, \u2018national emergency,\u2019 Sept. 5, 2023\nBurning Man Traffic post on X, Sept. 2, 2023\nBurning Man Traffic post on X, Sept. 2, 2023\nCenters for Medicare and Medicaid Services, Current emergencies, accessed Sept. 5, 2023\nFEMA, Declared Disasters, accessed Sept. 5, 2023\nThe White House, Statements and releases, accessed Sept. 5, 2023\nNevada Governor Joe Lombardo, Governor Lombardo Declares State of Emergency Due to Hurricane Hilary, Aug. 20, 2023\n\nSource: https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/\nTitle: \nContent: By early afternoon Sept. 4, the ban on exiting the Burning Man site expired, according to the event website. Some people waited in traffic for hours when leaving Sept. 5, the Reno Gazette-Journal reported. Organizers did not respond to PolitiFact\u2019s request for comment.\nOn Aug. 20, Nevada Gov. Joe Lombardo declared a state of emergency because severe weather related to Hurricane Hilary was predicted statewide. Lombardo encouraged people in Nevada to prepare for flooding and washouts and said FEMA had deployed staff to support the state\u2019s response.\u00a0\nForecasters predicted that as Hurricane Hilary dissipated it would drop significant rain on parts of Southern California and southern Nevada through the morning of Aug. 22.\u00a0\nBurning Man is held in northwestern Nevada; the rain and flooding concerns began Sept. 1. Lombardo\u2019s office did not respond to PolitiFact\u2019s request for comment about the Burning Man flooding.\u00a0\nOur ruling\n\nSource: https://apnews.com/article/burning-man-festival-flooding-entrance-closed-d6cd88ee009c6e1f6d2d92739ec1ca18\nTitle: \nContent: Burning Man flooding strands tens of thousands at Nevada site; authorities are investigating 1 death | AP News\nMenu\nMenu\nWorld\nU.S.\nElection 2024\nPolitics\nSports\nEntertainment\nBusiness\nScience\nFact Check\nOddities\nBe Well\nNewsletters\nVideo\nPhotography \nClimate\nHealth\nPersonal Finance\nAP Investigations\nTech\nLifestyle\nReligion\nAP Buyline Personal Finance\nAP Buyline Shopping\nPress Releases\nMy Account\n...\nWorld\nIsrael-Hamas War\n    \nRussia-Ukraine War\n    \nGlobal elections\n    \nAsia Pacific\n    \nLatin America\n    \nEurope\n    \nAfrica\n    \nMiddle East\n    \nChina\n    \nAustralia\n    \nU.S.\nElection 2024\nElection Results\n    \nDelegate Tracker\n    \nAP & Elections\n    \nGlobal elections\n    \nPolitics\nJoe Biden\n    \nElection 2024\n    \nCongress\n    \nSports\nMLB\n    \nNBA\n    \nNHL\n    \nNFL\n    \nSoccer\n    \nGolf\n    \nTennis\n    \nAuto Racing\n    \n2024 Paris Olympic Games\n    \nEntertainment\nMovie reviews\n    \nBook reviews\n    \nCelebrity\n    \nTelevision\n    \nMusic\n    \nBusiness\nInflation\n\nSource: https://www.knpr.org/news-88-9-knpr/2023-12-27/after-flooding-burning-man-to-go-on-as-planned-in-2024\nTitle: \nContent: Published\u00a0December 27, 2023 at 9:13 AM PST\n    \n \nFacebook\nTwitter\nLinkedIn\nEmail\nSteve Jurvetson/Flickr\nThe crowd gathered for at the annual Burning Man festival in Nevada's Black Rock desert. Tickets to the festival could be higher.\nBurning Man organizers don't foresee major changes in 2024 thanks to a hard-won passing grade for cleaning up this year's festival. Some feared their pledge to \u201cleave no trace\u201d might be too tall of a task after a rare rainstorm turned Nevada's high desert into a muddy quagmire, 110 miles north of Reno, delaying the departure of 80,000 revelers. Once out, organizers had six weeks to clean up under terms of a federal permit. By the smallest of margins, they passed the test last month, with a few adjustments recommended for the future. \nSponsor Message\nThe verdict from the BLM means Burning Man is in line to use federal land again next year.\nTags\nLocal News\nEnergy and EnvironmentLocal News\n \nFacebook\nTwitter\nLinkedIn\nEmail\nRick Andrews\n\nSource: https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/\nTitle: \nContent: The flooding temporarily stranded thousands of Burning Man attendees, who were told to shelter in place and conserve their supplies.\u00a0\nWe consulted federal websites and found no evidence the flooding at Burning Man was declared a national emergency. By early afternoon Sept. 4, the road to exit the event had been reopened.\nSee the sources for this fact-check\nHeavy rains in Nevada temporarily stranded thousands of people and put a damper on this year\u2019s Burning Man festival, but the situation didn\u2019t trigger a federal emergency response.\nThe event is a popular counterculture celebration that has attracted thousands of people to Nevada\u2019s Black Rock Desert for more than 30 years.\n",
    "Source: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: Burning Man flooding: What happened to stranded festivalgoers? - ABC News\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: ABC NewsVideoLiveShowsElections538ShopInterest Successfully AddedWe'll notify you here with news aboutTurn on desktop notifications for breaking stories about interest? OffOnLOG INStream onBurning Man flooding: What happened to stranded festivalgoers?Some 64,000 people were still on site Monday as the exodus began.ByKiara AlfonsecaSeptember 4, 2023, 8:49 PM2:49Attendees look at a double rainbow over flooding on a desert plain at the Burning Man festival in Black Rock City, Nevada, September 1, 2023.Julie Jammot/AFP via Getty ImagesTens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.The festival, held in the Black Rock Desert in northwestern Nevada, was attended by more than 70,000 people last year, and just as many were expected this year. Burning Man began on Aug. 28 and was scheduled to run through Sept. 4.One person died at the festival amid the unusual weather conditions, the\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: conserve food and water and avoid driving or operating any vehicle on the campgrounds.Attendees look at a double rainbow over flooding on a desert plain at the Burning Man festival in Black Rock City, Nevada, September 1, 2023.Julie Jammot/AFP via Getty ImagesAttendees were advised not to operate any generators or other electrical instruments standing in water.On Sunday, an afternoon drizzle compounded onto already poor conditions at the campgrounds. The main gate road was still impassable on Sunday night, and alternative exit routes have been planned for the expected exodus of attendees from festival grounds on Monday.As some attendees prepare to leave on foot, shuttle buses are running to assist in the exit.MORE: Idalia causes major flooding in CharlestonUpdates are being housed on the \"2023 Wet Playa Survival Guide\" created by event organizers.\"Burning Man is a community of people who are prepared to support one another,\" the guide read. \"We have come here knowing this is a place\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: and it does not take much rain to make the desert floor a mud bath.The downpour was followed by cooler temperatures and cloudy skies -- extending the drying-out process.However, better conditions on Monday are expected to bring \"a welcome chance to dry out,\" event organizers said.Structures stand in mud after heavy rain during the Burning Man festival in Black Rock City, Nevada, Sept. 1, 2023.Evi Airy/ReutersWhat did festivalgoers face?In response to the unusual weather, event organizers shut down traffic in or out of what is called Black Rock City -- where the festival is held in the desert -- including the local airport.Photos show festival grounds covered in muddy puddles, with some attendees braving the messy conditions.DJ Diplo claimed on X, formerly known as Twitter, that he and Chris Rock walked 5 miles in the mud before a fan picked them up.Attendees were asked to shelter in place, conserve food and water and avoid driving or operating any vehicle on the campgrounds.Attendees\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: at this time,\" according to the sheriff's office statement.Organizers lifted a driving ban as of 2 p.m. local time Monday, allowing \"exodus operations\" to begin, organizers said. They are, however, recommending that people delay their departures until Tuesday to avoid getting stuck in the mud.Approximately 64,000 people remain on site as of midday Monday, organizers said.Camps are set on a muddy desert plain at the Burning Man festival in Black Rock, City, Nevada, Sept. 2, 2023.Julie Jammot/AFP via Getty Images What happened?Black Rock Desert was faced with two to three months worth of rain in just a matter of hours on Friday, Sept. 1.On average, the area gets only 0.2 inches of rain or less in September -- but festivalgoers were met with up to 1 inch in some areas, in a desert that gets only about 5 to 6 inches of rain per year.This is typically the driest time of the year for the desert, and it does not take much rain to make the desert floor a mud bath.The downpour was followed by\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: are prepared to support one another,\" the guide read. \"We have come here knowing this is a place where we bring everything we need to survive. It is because of this that we are all well-prepared for a weather event like this.\"Organizers said they worked with \"agency collaborators on the local, county, state, tribal, and federal levels\" to prepare and coordinate a response to the weather conditions.MORE: These US regions will experience scorching temperatures for the remainder of Labor Day weekendOn Sunday, mobile cell trailers to boost cell service and charging stations were placed around the festival grounds amid the recovery efforts, according to organizers.Burning Man has been hosted for over 30 years, according to a statement from the organizers.In 2013, according to a blog post in the \"Burning Man Journal,\" a rainstorm similarly rolled in, unexpectedly \"trapping 160 people on the playa overnight.\"On Sunday, President Joe Biden said he was in contact with locals and that the\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: the playa overnight.\"On Sunday, President Joe Biden said he was in contact with locals and that the government \"ought to be getting everybody out of there.\"\"We're in contact with the local people,\" Biden told reporters.The White House recommended that event attendees \"listen to state and local officials, and event organizers.\"Top StoriesTrump trial: Stormy Daniels reacts to suggestion she was faking Trump sex story5 minutes agoExclusive: Mom speaks out after Air Force rescued son from cruise shipMay 9, 7:36 AMHere's what legal experts say Stormy Daniels' testimony could mean for Trump3 hours agoMiss Teen USA 2023 resigns 2 days after Miss USA gives up titleMay 8, 6:09 PM4 seriously hurt when Transair Boeing 737 skids off runway in Senegal1 hour agoABC News Live24/7 coverage of breaking news and live eventsABC News NetworkAbout Nielsen MeasurementChildren's Online Privacy PolicyContact UsDo Not Sell or Share My Personal InformationInterest-Based AdsPrivacy PolicyTerms of UseYour US\n\nSource: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331\nTitle: \nContent: to run through Sept. 4.One person died at the festival amid the unusual weather conditions, the Pershing County Sheriff's Office confirmed Sunday morning in a statement. The death is under investigation.On Monday, authorities confirmed the deceased had been identified as Leon Reece, 32. Pershing County Sheriff\u2019s Office deputies performed a preliminary investigation, and interviewed witnesses at the scene and medical responders. Reece's remains were sent to Washoe County Medical Examiner\u2019s Office for an autopsy. The results of the autopsy are pending toxicology results.Reece's death is still being investigated, authorities said Monday.\"The Pershing County Sheriff's Office is currently investigating a death which occurred during this rain event. The family has been notified and the death is under investigation. As this death is still under investigation, there is no further information available at this time,\" according to the sheriff's office statement.Organizers lifted a driving ban\n"
]




In [ ]:
from langchain.output_parsers import PydanticOutputParser
from llm_researcher.utils.validators import Subtopics

parser = PydanticOutputParser(pydantic_object=Subtopics)


In [ ]:
from llm_researcher.master.prompts import generate_subtopics_prompt
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
        template=generate_subtopics_prompt(),
        input_variables=["task", "data", "subtopics", "max_subtopics"],
        partial_variables={
                       "format_instructions": parser.get_format_instructions()},
        )


In [ ]:
from llm_researcher.config.config import Config
config = Config()
llm_provider = config.llm_provider(
    model=config.smart_llm_model,
    temperature=0,
    max_tokens=config.smart_token_limit)


model = llm_provider.llm
model

In [ ]:
report = """
# In-Depth Report on the 2023 Burning Man Floods

## Introduction

The Burning Man festival, an annual event known for its celebration of community, art, and self-expression, faced an unprecedented challenge in 2023 when heavy rainfall led to significant flooding at the event site in Nevada's Black Rock Desert. This report provides a comprehensive overview of the events that unfolded, the response from organizers and authorities, and the impact on attendees.

## The Onset of the Storm

The festival, which began on August 28, 2023, was attended by tens of thousands of participants. However, the event was disrupted when an unusual late-summer storm hit the area, resulting in two to three months' worth of rain falling in just 24 hours between Friday, September 1, and Saturday morning, September 2 ([NBC News](https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193)). The National Weather Service reported that up to 0.8 inches of rain fell during this period, which is about twice the average September rainfall for the region ([CNN](https://www.cnn.com/2023/09/05/us/burning-man-storms-shelter-exodus-tuesday/index.html)).

## Impact on Attendees and Infrastructure

The heavy rainfall transformed the desert into a muddy quagmire, making the ground impassable for vehicles and leaving approximately 70,000 attendees stranded ([CBS News](https://www.cbsnews.com/news/burning-man-death-investigation-70000-stranded-nevada-desert-flooding/)). The festival's infrastructure was severely affected, with reports of foot-deep mud and non-operational toilets ([CBS12](https://cbs12.com/news/nation-world/flooding-strands-thousands-at-burning-man-festival-site-death-under-investigation-black-rock-desert-nevada-playa-severe-weather-conditions-bureau-of-land-management)). The situation was exacerbated by the lack of working toilets and the difficulty in maintaining sanitary conditions.

## Response from Organizers and Authorities

In response to the flooding, Burning Man organizers advised attendees to "shelter in place" and conserve food and water. They also halted entry and exit from the area and urged people not to drive, as the desert floor had turned into thick clay ([CNN](https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html)). The organizers worked with local, county, state, tribal, and federal agencies to coordinate a response to the weather conditions. Mobile cell trailers were placed around the festival grounds to boost cell service, and charging stations were made available ([ABC News](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331)).

## The Death Investigation

Amid the chaos, there was a report of one death at the festival. The Pershing County Sheriff's Office confirmed that they were investigating the death, which occurred during the rain event. However, they clarified that the death was not caused by the rain itself. The identity of the deceased and the circumstances surrounding the death were not immediately released, as the investigation was ongoing ([CBS News](https://www.cbsnews.com/news/burning-man-death-investigation-70000-stranded-nevada-desert-flooding/)).

## The Aftermath and Recovery

As conditions improved, the ban on exiting the Burning Man site was lifted by early afternoon on September 4. Attendees were able to begin leaving the site, although many faced long waits in traffic. Organizers recommended delaying departures until Tuesday to avoid vehicles getting stuck in the mud. Approximately 64,000 people remained on site as of midday Monday ([ABC News](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331)).

## Misinformation and Clarifications

There were claims on social media that the flooding at Burning Man had triggered a national emergency declaration. However, these claims were found to be false. No evidence suggested that the federal government had declared a national emergency in response to the flooding at the festival ([PolitiFact](https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/)).

## Conclusion

The 2023 Burning Man festival was marked by an unexpected natural disaster that tested the resilience and preparedness of both the attendees and the organizers. Despite the severe weather conditions and the challenges they posed, the community came together to support one another through the crisis. The event's principle of "radical self-reliance" was put to the test, and the response demonstrated the strength and adaptability of the Burning Man community.

## References

- [NBC News](https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193)
- [CNN](https://www.cnn.com/2023/09/05/us/burning-man-storms-shelter-exodus-tuesday/index.html)
- [CBS News](https://www.cbsnews.com/news/burning-man-death-investigation-70000-stranded-nevada-desert-flooding/)
- [CBS12](https://cbs12.com/news/nation-world/flooding-strands-thousands-at-burning-man-festival-site-death-under-investigation-black-rock-desert-nevada-playa-severe-weather-conditions-bureau-of-land-management)
- [CNN](https://www.cnn.com/2023/09/02/us/burning-man-storms-shelter-black-rock-city/index.html)
- [ABC News](https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331)
- [PolitiFact](https://www.politifact.com/factchecks/2023/sep/06/instagram-posts/there-were-floods-there-was-mud-but-burning-man-sp/)
"""

In [ ]:
from llm_researcher.utils.write_md import write_md_to_pdf
await write_md_to_pdf(report)

In [ ]:
from copy import deepcopy

global_context_tmp = deepcopy(global_context)
global_urls_tmp = deepcopy(global_urls)
existing_headers_tmp = []
subtopic = subtopics[0]
report = await get_subtopic_report(query, subtopic, agent, role, global_context_tmp, global_urls_tmp, existing_headers_tmp)
report

In [ ]:
existing_headers_tmp

# Detail Report